# Paralelni algoritmi - treći projekat

- Projekat se radi individualno ili u paru

- Rok za predaju projekata je ponedeljak 28.12.2020. Biće potrebno da se studenti koji planiraju da urade i odbrane projekat prijave, do datuma i procedurom koji će naknadno biti objavljeni.

- Predaja projekata je putem emaila, programski kod bi trebalo da
bude prikacen uz sam email. Link ka Collab-u je opcion. Ukoliko dva
studenta rade u paru, dovoljno je da jedan od njih preda, pri cemu
drugog stavi u cc. E-mail treba da sadrzi sve podatke (ime, prezime,
broj indeksa) za oba studenta.

Koristeći PyCuda okurženje napisati CUDA program za (matrično) množenje matrica.

1. [5 bodova] Program koji vrši množenja matrica malih dimenzija (množenje se može izvršiti upotrebom jednom bloka niti)
2. [5 bodova] Probram koji vrši množenje matrica većih dimenzija, upotrebom većeg broja CUDA blokova.
3. [7 bodova] Ubrzati rešenje iz stavke 2 upotrebom deljene memorije (tako da niti jednog bloka prvo dovuku deo podataka u deljenu memeorju, a potom sve čitaju iz deljene memorije)
4. [8 bodova] Izmeniti rešenje iz tačke 3 tako da se pri množenju druga matrica transponuje ($Rezultat = A \cdot B^T$)

-----------------------------------------------------------------------------------------------

1. [5 bodova] Program koji vrši množenja matrica malih dimenzija (množenje se može izvršiti upotrebom jednom bloka niti)

In [ ]:
!pip install pycuda

In [ ]:
!nvidia-smi

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
                                                                            
mod = SourceModule(""" 
        __global__ void mnozenjeMatrica(float *a, float *b, float *c, 
                                        int widthA, int widthB){ 
              
              int idx_A = threadIdx.y * widthA;
              int idx_B = threadIdx.x;
              int idx_C = threadIdx.x + threadIdx.y * widthB;

              for(int i=0; i < widthA; i++){
                c[idx_C] += a[idx_A + i] * b[idx_B];
                idx_B += widthB;
              }
            }
        """)

In [ ]:
import numpy as np

a = np.random.randn(2, 3).astype(dtype=np.float32)
b = np.random.randn(3, 2).astype(dtype=np.float32)

a = np.round(a, 1)
b = np.round(b, 1)
print(a)
print("\n")
print(b)
print("\n")

ocekivano = a.dot(b)
c = np.zeros_like(ocekivano)

a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)

x = np.int(b.shape[1])#shape[1] kolone
y = np.int(a.shape[0])#shape[0] redovi

func = mod.get_function("mnozenjeMatrica")
func(a_gpu, b_gpu, c_gpu, np.int32(a.shape[1]), np.int32(b.shape[1]), #cast to np int
    block = (x, y, 1),
    grid = (1,1,1))

cuda.memcpy_dtoh(c, c_gpu)

ocekivano = np.round(ocekivano, 1)
c = np.round(c, 1)

print(ocekivano)
print("\n")
print(c)
print("\n")

print('Dobijen ocekivan rezultat: ', (c == ocekivano).all())

[[ 0.9  2.7  0.4]
 [-2.  -1.1  0.8]]


[[ 0.6  0.6]
 [-0.7  0. ]
 [-1.2 -0.4]]


[[-1.8  0.4]
 [-1.4 -1.5]]


[[-1.8  0.4]
 [-1.4 -1.5]]


Dobijen ocekivan rezultat:  True


2. [5 bodova] Probram koji vrši množenje matrica većih dimenzija, upotrebom većeg broja CUDA blokova.

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule(""" 
        __global__ void mnozenjeVecihMatrica(float *a, float *b, float *c, 
                                            int cols_B, int rows_A, int cols_A) {
              
              long idx_A = threadIdx.y + blockDim.y * blockIdx.y;
              long idx_B = blockDim.x * blockIdx.x + threadIdx.x;
              long idx_C = threadIdx.x + blockDim.x * blockIdx.x + threadIdx.y * cols_B + blockDim.y * blockIdx.y * cols_B;

              if(idx_B > cols_B || idx_A > rows_A) 
                return;
             
              for(int i = 0; i < cols_A; i++){
                c[idx_C] += a[idx_A * cols_A + i] * b[idx_B];
                
                idx_B += cols_B;
              }
            }
        """)

In [ ]:
import numpy as np
import math

a = np.random.randn(30, 96).astype(dtype=np.float32)
b = np.random.randn(96, 100).astype(dtype=np.float32)

ocekivano = a.dot(b)
c = np.zeros_like(ocekivano)

a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)

x = np.int(b.shape[1])#shape[1] kolone
y = np.int(a.shape[0])#shape[0] redovi

func = mod.get_function("mnozenjeVecihMatrica")
func(a_gpu, b_gpu, c_gpu, np.int32(b.shape[1]), np.int32(a.shape[0]), np.int32(a.shape[1]),
     block = (32, 32, 1), 
     grid = (math.ceil(b.shape[1]/32), math.ceil(a.shape[0]/32), 1))

cuda.memcpy_dtoh(c, c_gpu)

ocekivano = np.round(ocekivano, 2)
c = np.round(c, 2)
print(ocekivano)
print("\n")
print(c)
print("Dobijen ocekivan rezultat: ", (c == ocekivano).all())

[[ -7.22  -7.29 -18.95 ...  -2.23   5.32 -13.74]
 [-11.87   6.91 -10.74 ...  -7.09   3.62   3.32]
 [ 12.99  19.43   4.84 ...  -7.8   -0.32  -1.71]
 ...
 [  3.77   1.72 -25.68 ...   3.42   1.9    9.3 ]
 [ -6.5   -2.27  17.21 ...   2.72  -8.13  -2.47]
 [ -6.24   7.29   4.02 ... -17.17  -0.26  13.43]]


[[ -7.22  -7.29 -18.95 ...  -2.23   5.32 -13.74]
 [-11.87   6.91 -10.74 ...  -7.09   3.62   3.32]
 [ 12.99  19.43   4.84 ...  -7.8   -0.32  -1.71]
 ...
 [  3.77   1.72 -25.68 ...   3.42   1.9    9.3 ]
 [ -6.5   -2.27  17.21 ...   2.72  -8.13  -2.47]
 [ -6.24   7.29   4.02 ... -17.17  -0.26  13.43]]
Dobijen ocekivan rezultat:  True


3. [7 bodova] Ubrzati rešenje iz stavke 2 upotrebom deljene memorije (tako da niti jednog bloka prvo dovuku deo podataka u deljenu memeorju, a potom sve čitaju iz deljene memorije)


In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule  

mod = SourceModule(""" 
        __global__ void mnozenjeDeljenaMemorija(float *a, float *b, float *c,
                                                int rows_A, int cols_A,
                                                int rows_B, int cols_B,
                                                int rows_C, int cols_C) {
              
              #define SIZE 32

              float matrix_c = 0;

              int row = blockIdx.y * blockDim.y + threadIdx.y;
              int col = blockIdx.x * blockDim.x + threadIdx.x;
              int idx = row * cols_C + col;

              __shared__ float shared_a[SIZE][SIZE];
              __shared__ float shared_b[SIZE][SIZE];

              for(int i = 0; i < (cols_A - 1) / blockDim.x + 1; i++){

                if(i * blockDim.y + threadIdx.x < cols_A && row < rows_A){
                  shared_a[threadIdx.y][threadIdx.x] = a[row * cols_A + i * blockDim.y + threadIdx.x];
                } else {
                  shared_a[threadIdx.y][threadIdx.x] = 0.0;
                }

                if(i * blockDim.y + threadIdx.y < rows_B && col < cols_B){
                  shared_b[threadIdx.y][threadIdx.x] = b[(i * blockDim.y + threadIdx.y) * cols_B + col];
                } else {
                  shared_b[threadIdx.y][threadIdx.x] = 0.0;
                }

                __syncthreads();

                for(int j = 0 ; j < blockDim.y ; j++){
                  matrix_c += shared_a[threadIdx.y][j] * shared_b[j][threadIdx.x];
                }

                __syncthreads();

                if(row < rows_C && col < cols_C){
                  c[idx] = matrix_c;
                }                
              } 
            }
        """)

In [ ]:
import numpy as np
import math

a = np.random.randn(70,100).astype(dtype=np.float32)
b = np.random.randn(100,70).astype(dtype=np.float32)

a = np.round(a, 4)
b = np.round(b, 4)
print(a)
print("\n")
print(b)
print("\n")

res = a.dot(b)
c = np.zeros_like(res)

a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)

func = mod.get_function("mnozenjeDeljenaMemorija")
func(a_gpu, b_gpu, c_gpu,np.int32(a.shape[0]),np.int32(a.shape[1]),np.int32(b.shape[0]),np.int32(b.shape[1]),np.int32(a.shape[0]),np.int32(b.shape[1]),
     block=(32, 32, 1), 
     grid=(math.ceil(b.shape[1]/32), math.ceil(a.shape[0]/32), 1))

cuda.memcpy_dtoh(c, c_gpu)

res = np.round(res, 2)
c = np.round(c, 2)

print(res)
print("\n")
print(c)
print("Dobijen ocekivan rezultat: ", (c == res).all())

[[-0.0509 -1.187  -0.9216 ... -0.4157  0.1601  0.436 ]
 [-0.8106 -0.6768 -1.1292 ... -1.5553 -1.9249  0.7436]
 [-0.6199 -0.3391 -1.2286 ... -0.3065 -0.0318  0.8306]
 ...
 [ 1.5367  0.3416 -0.3264 ... -1.0204 -0.3482  0.1431]
 [ 0.351  -1.0746 -1.5246 ... -1.083   0.2403  1.943 ]
 [ 0.7366 -3.3159  0.0729 ... -1.5974  0.6409  0.5684]]


[[ 1.6146 -0.1745 -0.2496 ...  0.0815 -0.2947 -0.5753]
 [-0.3724 -0.6016 -1.5025 ...  1.4752 -0.6094  1.2214]
 [-0.7612 -1.5029  1.2835 ...  1.5504  0.1913 -0.0488]
 ...
 [ 0.6037 -0.0685  0.2696 ... -0.7147 -1.5343  0.3015]
 [ 0.7721 -0.2098 -0.4891 ... -1.3526  0.4104 -1.5785]
 [-0.9243  0.2099  0.9388 ... -0.3433  1.1676  0.5311]]


[[ -1.16   6.18  -5.08 ...  15.96  -6.15 -15.9 ]
 [ 10.7    4.27  -2.66 ... -14.01  -7.93  -3.56]
 [  2.64  -1.59  -4.95 ...  13.78  12.4   -4.62]
 ...
 [ -6.69   4.38  16.78 ...  -4.53   5.81   6.43]
 [ -0.4   -2.91   3.48 ... -13.38  -1.     6.63]
 [-13.78   5.73  10.58 ...  -8.08  13.57   0.43]]


[[ -1.16   6.18  -5.08

4. [8 bodova] Izmeniti rešenje iz tačke 3 tako da se pri množenju druga matrica transponuje ($Rezultat = A \cdot B^T$)



In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule(""" 
        __global__ void mnozenjeTransponovanje(float *a,float *b, float *c,
                                                                int rows_A, int cols_A,
                                                                int rows_B, int cols_B,
                                                                int rows_C, int cols_C) {
              
              #define SIZE 32

              float matrix_c = 0;

              int row = blockIdx.y * blockDim.y + threadIdx.y;
              int col = blockIdx.x * blockDim.x + threadIdx.x;
              int idx = row * cols_C + col;

              __shared__ float shared_a[SIZE][SIZE];
              __shared__ float shared_b[SIZE][SIZE];

              for(int i = 0; i < (cols_A - 1) / blockDim.x + 1; i++){
                if(i * blockDim.y + threadIdx.x < cols_A && row < rows_A){
                  shared_a[threadIdx.y][threadIdx.x] = a[row * cols_A + i * blockDim.y + threadIdx.x];
                } else {
                  shared_a[threadIdx.y][threadIdx.x] = 0.0;
                }

                if(i * blockDim.y + threadIdx.y < rows_B && col < rows_B){
                  shared_b[threadIdx.y][threadIdx.x] = b[i * blockDim.y + threadIdx.y + cols_B * col];
                } else {
                  shared_b[threadIdx.y][threadIdx.x] = 0.0;
                }
               
                __syncthreads();

                for(int j = 0 ; j < blockDim.y; j++){
                  matrix_c += shared_a[threadIdx.y][j] * shared_b[j][threadIdx.x];
                }

                __syncthreads();

                if(row < rows_C && col < cols_C){
                  c[idx] = matrix_c;
                }                 
              } 
            }
        """)

In [ ]:
import numpy as np
import math

a = np.random.randn(30,96).astype(dtype=np.float32)
b = np.random.randn(100,96).astype(dtype=np.float32)

tran = np.transpose(b)
res = a.dot(tran)

c = np.zeros_like(res)

a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(a_gpu,a)
cuda.memcpy_htod(b_gpu,b)
cuda.memcpy_htod(c_gpu,c)

func = mod.get_function("mnozenjeTransponovanje")
func(a_gpu, b_gpu, c_gpu,np.int32(a.shape[0]),np.int32(a.shape[1]),np.int32(b.shape[0]),np.int32(b.shape[1]),np.int32(a.shape[0]),np.int32(b.shape[0]),
     block=(32, 32, 1), 
     grid=(math.ceil(np.int32(b.shape[0])/32), math.ceil(np.int32(a.shape[0])/32), 1))

cuda.memcpy_dtoh(c,c_gpu)

res = np.round(res, 2)
c = np.round(c, 2)

print(res)
print("\n")
print(c)
print("Dobijen ocekivan rezultat: ",(c==res).all())

[[ 10.7    5.2  -14.07 ...  -6.66   7.84  -9.25]
 [ -5.08  18.34   2.18 ...  -1.52  -1.92   5.17]
 [ -1.29   3.62  -8.64 ...   0.84  20.06  18.97]
 ...
 [ -9.08 -12.43  -4.06 ... -11.47  -2.57  13.75]
 [ 13.55   9.11  -1.64 ...  -1.42 -12.86   0.83]
 [-10.79   3.84   3.74 ...  12.27  11.18   0.72]]


[[ 10.7    5.2  -14.07 ...  -6.66   7.84  -9.25]
 [ -5.08  18.34   2.18 ...  -1.52  -1.92   5.17]
 [ -1.29   3.62  -8.64 ...   0.84  20.06  18.97]
 ...
 [ -9.08 -12.43  -4.06 ... -11.47  -2.57  13.75]
 [ 13.55   9.11  -1.64 ...  -1.42 -12.86   0.83]
 [-10.79   3.84   3.74 ...  12.27  11.18   0.72]]
Dobijen ocekivan rezultat:  True
